<a href="https://colab.research.google.com/github/hanna56/Teddy/blob/main/teddy_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

     |████████████████████████████████| 46.9MB 62kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 32.0MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595695 sha256=3e2a8be90e22a34ab290c8072998cfe6cb005b3315be4c4114108ea51d82255d
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 31.1MB/s 
     |████████████████████████████████| 757kB 31.5MB/s 
     |████████████████████████████████| 3.0MB 41.1MB/s 
     |████████████████████████████████| 901kB 43.0MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-722_krmt
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-722_krmt
  Created wheel for kobe

In [2]:
bertmodel, vocab = get_pytorch_kobert_model()

from google.colab import drive
drive.mount('/content/drive/')

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
Mounted at /content/drive/


In [5]:
# 학습용 데이터셋 불러오기
import pandas as pd
dataset = pd.read_excel('/content/drive/My Drive/Teddy 프로젝트/Bert/final.xlsx')
dataset.head()

,sentence,label
0,이런 기분은 느끼는 건 대학시절 만났던 첫사랑과의 연애 이후 처음이었다,사랑
1,일일이 증명하려 애쓰지 않아도 내가 어떤 사람인지 이해하는 연인이 있었다는 건 어설...,사랑
2,그때 왜 자신의 고백을 거절한 거였냐고 정말로 그냥 소중한 친구였기 때문이었냐고 어...,사랑
3,나는 열성팬도 아니지만 그가 무엇 이든지 간에 여전히 그가 그립고 좋다,사랑
4,로맨스 영화나 노래가 너무도 내 얘기 같은 걸 보니이런 걸 사랑이라고 하나 봐요,사랑


In [6]:
#데이터 총 개수
print(len(dataset))

33912


In [7]:
# 숫자로 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(dataset['label'])
dataset['label'] = encoder.transform(dataset['label'])
dataset.head()

# 매핑 ex) {0: '공포', 1: '기쁨', 2: '분노', 3: '사랑', 4: '슬픔'}
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: '공포', 1: '기쁨', 2: '분노', 3: '사랑', 4: '슬픔'}

In [8]:
#value error가 발생하여 list 형식으로 변경해주었다.
lst=[list(dataset.iloc[i]) for i in range(len(dataset))]
print(lst[:10])

[['이런 기분은 느끼는 건 대학시절 만났던 첫사랑과의 연애 이후 처음이었다', 3], ['일일이 증명하려 애쓰지 않아도 내가 어떤 사람인지 이해하는 연인이 있었다는 건 어설픈 이십 대 청춘에겐 과분한 축복이었다 는 멋진 여자가 되어있었다', 3], ['그때 왜 자신의 고백을 거절한 거였냐고 정말로 그냥 소중한 친구였기 때문이었냐고 어른이 된 지 한참이 지난 탓인지 나 역시 그때의 나에게 공감할 수는 없었다', 3], ['나는 열성팬도 아니지만 그가 무엇 이든지 간에 여전히 그가 그립고 좋다', 3], ['로맨스 영화나 노래가 너무도 내 얘기 같은 걸 보니이런 걸 사랑이라고 하나 봐요', 3], ['그런 내가 누군가에게 좋아한다고 고백 을 한다구 결코 나답지 않은 행동이라 생각했죠', 3], ['세월이 흘러 언제나 네 짝을 찾아 곁에 두고 살지 못한다 해도 너 어릴 적 엄마가 근무 중에 너를 가끔 생각하고 네 옆자리에 앉은 모습을 상상하며 기도로서 너를 지켜 주었던 것과 같이 항상 널 위해 또 네가 사랑하는 가족을 위해 엄만 항상 기도 할 거야 언제나 사랑으로 충만한 엄마가 되어 줄게', 3], ['뽀얀 얼굴에 오목조목 예쁘게 생긴 여자 아이 가 우리 반으로 전학을 왔다 는 발랄하고 톡톡 튀어서 친구들이 좋아했다', 3], ['그 오빠 말인 즉 본인은 그의 누나를 좋아한다고 그녀의 동생이 엘레강스 너를 좋아하니 잘 되었으면 좋겠다고 지나와서 돌아보니 이성에 눈을 뜨고 관심을 보이기 시작한 한낱 귀여운 시절에불과할 뿐이다', 3], ['돌이켜 보건대 내가 이성으로 좋아했던 남자들은 모두 형이고 직장생활을 하면서 파트너십이 좋았던 사람들도 형이다', 3]]


In [9]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(lst, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 27129
test shape is: 6783


In [10]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

using cached model


In [11]:
# Setting parameters
max_len = 128 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 100 
learning_rate = 5e-5

In [12]:
data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

In [13]:
data_train[0]

(array([   2, 3403, 5570, 3403, 5439,  930, 3093, 7005, 7609, 1550, 3220,
        5907, 7010,  517, 6538, 6150, 3836, 6887, 1282, 7216, 5887, 4427,
        4204, 7074, 5702,  517, 6894, 5848, 7096, 5859,  517, 5960,  517,
        5849, 7096, 5439, 5039, 5642, 7078, 3952, 7276, 5439,  777, 6150,
        6896, 1460, 1947, 6667, 3454, 7096, 5804, 3327, 3454, 7086, 1535,
        1735, 5400,  517, 5960,  517, 5621, 7899, 6325, 5377, 2196, 2573,
        6553, 3836, 5760, 5400, 1947, 5859,  517, 7242, 1513,  517,  492,
         492, 6824, 7668, 3697, 7280,  517, 7601, 6312, 7095, 3349, 1185,
        5377, 1266, 5760, 3977, 6079, 2643, 6828,  517, 6029, 6896, 6770,
         517, 6870, 5538, 6054, 2302, 7609,  517, 7753, 6050, 7442,  517,
        5968, 5411, 6553, 4982,  517, 6720, 6183, 5495,  517,  493,  493,
        1460, 1967, 7143,  517,    0,    3,    1], dtype=int32),
 array(127, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0

In [14]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
model = BERTClassifier(bertmodel, dr_rate=0.5).cuda()

In [17]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [18]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [19]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [20]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [21]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [22]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().cuda()
        segment_ids = segment_ids.long().cuda()
        valid_length= valid_length
        label = label.long().cuda()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval() # 평가 모드로 변경

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().cuda()
        segment_ids = segment_ids.long().cuda()
        valid_length= valid_length
        label = label.long().cuda()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


epoch 1 batch id 1 loss 1.651896595954895 train acc 0.1875
epoch 1 batch id 101 loss 1.4348409175872803 train acc 0.2614480198019802
epoch 1 batch id 201 loss 0.48965445160865784 train acc 0.4839863184079602
epoch 1 batch id 301 loss 0.2732069194316864 train acc 0.6312292358803987
epoch 1 batch id 401 loss 0.08950553834438324 train acc 0.7095542394014963
epoch 1 batch id 501 loss 0.043687496334314346 train acc 0.7559256487025948
epoch 1 batch id 601 loss 0.031750429421663284 train acc 0.7880095673876872
epoch 1 batch id 701 loss 0.15248537063598633 train acc 0.8119204707560628
epoch 1 batch id 801 loss 0.0360688716173172 train acc 0.8304853308364545

epoch 1 train acc 0.838111733490566


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.9553313298843579


epoch 2 batch id 1 loss 0.01860571652650833 train acc 1.0
epoch 2 batch id 101 loss 0.16180865466594696 train acc 0.9647277227722773
epoch 2 batch id 201 loss 0.19325746595859528 train acc 0.9665733830845771
epoch 2 batch id 301 loss 0.18889448046684265 train acc 0.970203488372093
epoch 2 batch id 401 loss 0.029787229374051094 train acc 0.9711658354114713
epoch 2 batch id 501 loss 0.0064850435592234135 train acc 0.9718063872255489
epoch 2 batch id 601 loss 0.0033544222824275494 train acc 0.9727017470881864
epoch 2 batch id 701 loss 0.0461580753326416 train acc 0.9737428673323824
epoch 2 batch id 801 loss 0.006378286052495241 train acc 0.9744850187265918

epoch 2 train acc 0.9750884433962265


...


epoch 3 test acc 0.9752358490566038


epoch 4 batch id 1 loss 0.0010424766223877668 train acc 1.0
epoch 4 batch id 101 loss 0.003601626493036747 train acc 0.9938118811881188
epoch 4 batch id 201 loss 0.006661640480160713 train acc 0.9939365671641791
epoch 4 batch id 301 loss 0.02291034907102585 train acc 0.9947051495016611
epoch 4 batch id 401 loss 0.0008514715591445565 train acc 0.9950124688279302
epoch 4 batch id 501 loss 0.0006712668691761792 train acc 0.9956337325349301
epoch 4 batch id 601 loss 0.0008729207911528647 train acc 0.9958402662229617
epoch 4 batch id 701 loss 0.0009063892066478729 train acc 0.9959878744650499
epoch 4 batch id 801 loss 0.0005615938571281731 train acc 0.9960986267166042

epoch 4 train acc 0.9962411556603774



epoch 4 test acc 0.9818691037735849


epoch 5 batch id 1 loss 0.0007571045425720513 train acc 1.0
epoch 5 batch id 101 loss 0.12551972270011902 train acc 0.9987623762376238
epoch 5 batch id 201 loss 0.0009561189217492938 train acc 0.9992226368159204
epoch 5 batch id 301 loss 0.0005219888407737017 train acc 0.998857973421927
epoch 5 batch id 401 loss 0.0005244599306024611 train acc 0.9988310473815462
epoch 5 batch id 501 loss 0.0005776105099357665 train acc 0.998814870259481
epoch 5 batch id 601 loss 0.0005644260672852397 train acc 0.9988040765391015
epoch 5 batch id 701 loss 0.0006622375803999603 train acc 0.998796362339515
epoch 5 batch id 801 loss 0.0005311741260811687 train acc 0.9989076154806492

epoch 5 train acc 0.9989313089622641



epoch 5 test acc 0.9849646226415094


In [23]:
#새로운 데이터로 테스트
test_sen=[['그의 말로 나는 그만 울어버리고 말았다',4], ['우리 아기 엄마가 많이 사랑해',3],
          ['그의 안광에 그만 나는 소름이 끼쳤다',0],['사고 싶은 물건이 할인을 많이 해서 기분이 짜릿했다',1],
          ['그 새끼 진짜 짜증난다',2]]

In [24]:
data_test_sen = BERTDataset(test_sen, 0, 1, tok, max_len, True, False)
test_sen_dataloader = torch.utils.data.DataLoader(data_test_sen, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [25]:
test_sen_acc=0.0

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_sen_dataloader)):
  token_ids = token_ids.long().cuda()
  segment_ids = segment_ids.long().cuda()
  valid_length= valid_length
  label = label.long().cuda()
  out = model(token_ids, valid_length, segment_ids)
  print(out)
  
  test_sen_acc += calc_accuracy(out, label)
print("test_sen_acc {}".format(test_sen_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


tensor([[-1.5103, -2.5566, -2.2952, -2.2119,  7.2365],
        [-2.0434, -1.8831, -1.6721,  8.2456, -1.8851],
        [ 7.6741, -1.9886, -1.5429, -1.6806, -1.5053],
        [-2.5101,  6.4901,  1.1190, -2.2611, -2.4149],
        [-1.8246, -1.8470,  7.5584, -1.4505, -1.4890]], device='cuda:0',
       grad_fn=<AddmmBackward>)

test_sen_acc 1.0


In [ ]:
#모델 저장
# torch.save(model.state_dict(), '/content/drive/My Drive/teddy_project/full_model.pt')